In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, FloatType
import librosa
import os
import gc
import audioread
from audioread.exceptions import NoBackendError

In [2]:
def extract_mfcc(audio_path):
    try:
        audio, sample_rate = librosa.load(audio_path, sr=None)
        mfcc_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=100).mean(axis=1)
        return mfcc_features.tolist()
    except NoBackendError as e:
        print("No suitable audio backend found:", e)

In [3]:
extract_mfcc_udf = udf(extract_mfcc, ArrayType(FloatType()))

def process_and_save_to_mongo(file_list, spark_session):
    df_audio_paths = spark_session.createDataFrame(file_list, StringType()).toDF("audio_path")
    df_mfcc_features = df_audio_paths.repartition(100).withColumn("mfcc_features", extract_mfcc_udf(df_audio_paths["audio_path"]))
    df_mfcc_features.show(truncate=False)
    df_mfcc_features.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()
    del df_audio_paths, df_mfcc_features
    gc.collect()

In [4]:
input_uri = "mongodb://localhost:27017/bda.audio_features"
output_uri = "mongodb://localhost:27017/bda.audio_features"

In [5]:
spark = SparkSession.builder \
    .appName("AudioFeaturesExtraction") \
    .config("spark.mongodb.input.uri", input_uri) \
    .config("spark.mongodb.output.uri", output_uri) \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2") \
    .getOrCreate()

24/05/11 20:54:32 WARN Utils: Your hostname, moaz-HP-ProBook-440-G5 resolves to a loopback address: 127.0.1.1; using 192.168.10.5 instead (on interface wlp2s0)
24/05/11 20:54:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/moaz/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/moaz/.ivy2/cache
The jars for the packages stored in: /home/moaz/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-95884fe4-9fb7-4fde-a5dc-52151347a8c9;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 409ms :: artifacts dl 21ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |

In [6]:
folder_path = "DataSet"
all_folders = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
audio_files = []
for folder in all_folders:
    for file_name in os.listdir(folder):
        if file_name.endswith('.mp3'):
            audio_files.append(os.path.join(folder, file_name))

audio_files

['DataSet/067/067985.mp3',
 'DataSet/067/067260.mp3',
 'DataSet/067/067766.mp3',
 'DataSet/067/067729.mp3',
 'DataSet/067/067102.mp3',
 'DataSet/067/067280.mp3',
 'DataSet/067/067253.mp3',
 'DataSet/067/067425.mp3',
 'DataSet/067/067170.mp3',
 'DataSet/067/067549.mp3',
 'DataSet/067/067014.mp3',
 'DataSet/067/067590.mp3',
 'DataSet/067/067509.mp3',
 'DataSet/067/067237.mp3',
 'DataSet/067/067627.mp3',
 'DataSet/067/067079.mp3',
 'DataSet/067/067175.mp3',
 'DataSet/067/067165.mp3',
 'DataSet/067/067778.mp3',
 'DataSet/067/067673.mp3',
 'DataSet/067/067649.mp3',
 'DataSet/067/067395.mp3',
 'DataSet/067/067193.mp3',
 'DataSet/067/067761.mp3',
 'DataSet/067/067586.mp3',
 'DataSet/067/067263.mp3',
 'DataSet/067/067523.mp3',
 'DataSet/067/067857.mp3',
 'DataSet/067/067765.mp3',
 'DataSet/067/067589.mp3',
 'DataSet/067/067461.mp3',
 'DataSet/067/067110.mp3',
 'DataSet/067/067101.mp3',
 'DataSet/067/067723.mp3',
 'DataSet/067/067273.mp3',
 'DataSet/067/067174.mp3',
 'DataSet/067/067000.mp3',
 

In [7]:
chunk_size = 3000
for i in range(0, len(audio_files), chunk_size):
    chunk_files = audio_files[i:i+chunk_size]
    process_and_save_to_mongo(chunk_files, spark)

spark.stop()

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
/home/moaz/.local/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
No suitable audio backend found: 
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
/home/moaz/.local/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
No suitable au

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (2560) too large for available bit count (2336)
[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (1536) too large for available bit count (1048)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
No suitable audio backend found: 
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!0]
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
No suitable audio backend found: 
[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (2560) too large for available bit count (2336)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame st

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

...

[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!0]
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
/home/moaz/.local/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
No suitable audio backend found: 
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
No suitable audio backend found: 
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
No suitable audio backend found: 
[src/libmpg123/layer3.c:INT123_do_layer3():1

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
/home/moaz/.local/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
No suitable audio backend found: 
[src/libmpg123/layer3.c:INT123_do_layer3():1771] error: part2_3_length (1632) too large for available bit count (1568)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
No suitable audio backend found: 
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_4392/1519328582.

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!0]
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
No suitable audio backend found: 
/tmp/ipykernel_4392/1519328582.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
